# Gotta classify 'em all!

### Predict the types of Pokémon based on their attributes.

In the following, you have access to the vital statistics for the first 6 generations of Pokémon. Your (very open ended!) task is to see what to extent it is possible to predict the primary type (the `Type_1` field) of a Pokémon given its other vital statistics. This is a multiclass classification problem.

### Hints and tips:

* The package `sklearn` is the industry standard for ML algorithms that can be used out of the box quickly- you should use it. https://scikit-learn.org/stable/supervised_learning.html#supervised-learning
    * Beginning with something simple like Logistic Regression or a Decision Tree is encouraged to establish a performance baseline.
* The dataset features a large number of columns, many of which are likely redundant and will not contain any predictive power for the task at hand. 
    * Consider preselecting a small number of 'obvious' columns which your misspent youth (sorry, 'domain expertise') tells you are likely to contain a lot of predictice power. 
    * Get something up and running with these first (fewer features -> less time feature engineering) and then circle back to incorporate more features into your model once this is done.
    * Doing this iteratively rather than 'all at once' is good from a client facing point of view (having some kind of concrete result to discuss early is always a good thing), and will also help you see what is going on more quickly.
* The dataset consists of a mixture of categorical and continuous variables. 
    * Categorical variables will need to be converted to numeric indicator values before being passed into `sklearn` classifiers- you can use the `pandas` function `pd.get_dummies` for this.
    * Dummying lots of categorical features with lots of possible values can quickly result in a very large, sparse, feature space. Beginning with only a small subset of features will help here.
* Some of the columns in your dataset have null values in them:
    * It's wise to ignore these at a first pass in order to get up and running quickly, but they might contain information that could improve your model.
    * In this situation, does it make more sense to impute these null values with something like the mean or mode value of the non-null entries, to convert them to a dummy variable, or to incorporate them into another column somehow?
* There are `18` different `Type_1` values. Are these values distributed evenly across the dataset? It might make sense to focus your initial efforts on identifying only a subset of these in order to avoid getting bogged down with small data issues at the start.
* In order to evaluate the performance of your model, you will need to perform a train/test split (use `sklearn.model_selection.train_test_split`).
    * Research what it means to stratify your train/test split with respect to the target variable. It is a good idea to do so here in order to guarantee that the performance metrics you quote are relatively stable. You should be aware that doing this in situations where you can't be sure that the class balance breakdown 'in the wild' is the same as in your dataset will result in biased estimates of the performance of your model.
    * Train/test splits feature a frustrating tradeoff: you want as much data as possible in your train set to build the best possible model, but you also want lots of data in your test set to evaluate its performance accurately. Cross validation is a computationally intensive way to have your cake and eat it in this scenario: see `sklearn.model_selection.cross_val_score`.
* You will need to consider how to evaluate the performance of your classifier: the accuracy score is the simplest metric to quote, but plotting a full confusion matrix will give you significantly more insight into how your model is performing and where it could be improved.

In [1]:
# Feel free to import more packages (i.e., numpy, sklearn packages) as required.
import pandas as pd
import matplotlib.pyplot as plt

# Magic command to make plots render inline/underneath cells in Jupyter notebooks.
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [13]:
fpath = "https://s3-eu-west-1.amazonaws.com/faculty-client-teaching-materials/non-linear-algorithms/pokemon.csv"

In [14]:
df = pd.read_csv(fpath)
# list(df)

In [15]:
df.head()

,Number,Name,Type_1,Type_2,Total,HP,Attack,Defense,Sp_Atk,Sp_Def,...,hasGender,Pr_Male,Egg_Group_1,Egg_Group_2,hasMegaEvolution,Height_m,Weight_kg,Catch_Rate,Body_Style,train
0,239,Elekid,Electric,NaN,360,45,63,37,65,55,...,True,0.750,Undiscovered,NaN,False,0.61,23.5,45,bipedal_tailless,1
1,74,Geodude,Rock,Ground,300,40,80,100,30,30,...,True,0.500,Mineral,NaN,False,0.41,20.0,255,head_arms,1
2,515,Panpour,Water,NaN,316,50,53,48,53,48,...,True,0.875,Field,NaN,False,0.61,13.5,190,bipedal_tailed,1
3,535,Tympole,Water,NaN,294,50,50,40,50,40,...,True,0.500,Water_1,NaN,False,0.51,4.5,255,with_fins,1
4,441,Chatot,Normal,Flying,411,76,65,45,92,42,...,True,0.500,Flying,NaN,False,0.51,1.9,30,two_wings,1


In [5]:
df["Type_1"].unique()

array(['Electric', 'Rock', 'Water', 'Normal', 'Steel', 'Poison', 'Grass',
       'Ice', 'Ground', 'Fire', 'Psychic', 'Dark', 'Fighting', 'Dragon',
       'Bug', 'Flying', 'Ghost', 'Fairy'], dtype=object)

In [6]:
mostcommon = df["Type_1"].value_counts()

In [7]:
my_types = mostcommon.iloc[:19].index
my_features = [
    "Total",
    "HP",
    "Attack",
    "Defense",
    "Sp_Atk",
    "Sp_Def",
    "hasMegaEvolution",
    "Egg_Group_1",
]
my_features = [
    "Total",
    "HP",
    "Attack",
    "Defense",
    "Sp_Atk",
    "Sp_Def",
    "Egg_Group_1",
]

In [8]:
df.loc[df["Type_1"].isin(my_types), ["Type_1"] + my_features]

,Type_1,Total,HP,Attack,Defense,Sp_Atk,Sp_Def,Egg_Group_1
0,Electric,360,45,63,37,65,55,Undiscovered
1,Rock,300,40,80,100,30,30,Mineral
2,Water,316,50,53,48,53,48,Field
3,Water,294,50,50,40,50,40,Water_1
4,Normal,411,76,65,45,92,42,Flying
...,...,...,...,...,...,...,...,...
716,Grass,489,74,94,131,54,116,Grass
717,Steel,300,57,24,86,24,86,Mineral
718,Dark,288,53,54,53,37,46,Water_1
719,Rock,355,30,80,90,55,45,Water_1


In [10]:
subset = df["Type_1"].isin(my_types)

In [11]:
X = df[subset][my_features]
y = df[subset]["Type_1"]
X = pd.get_dummies(X, dtype=float)
X = X.dropna()

In [12]:
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=0, stratify=y
)
logisticRegr = LogisticRegression()
logisticRegr.fit(x_train, y_train)
y_pred = logisticRegr.predict(x_test)
score = logisticRegr.score(x_test, y_test)
print(score)

0.273972602739726


/opt/anaconda/envs/Python3/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [13]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         Bug       0.00      0.00      0.00         6
        Dark       0.00      0.00      0.00         3
      Dragon       0.00      0.00      0.00         2
    Electric       0.50      0.25      0.33         4
       Fairy       0.00      0.00      0.00         2
    Fighting       0.00      0.00      0.00         3
        Fire       0.25      0.20      0.22         5
       Ghost       0.00      0.00      0.00         2
       Grass       0.33      0.14      0.20         7
      Ground       0.00      0.00      0.00         3
         Ice       0.00      0.00      0.00         2
      Normal       0.33      0.67      0.44         9
      Poison       0.00      0.00      0.00         3
     Psychic       0.67      0.80      0.73         5
        Rock       0.38      0.75      0.50         4
       Steel       0.00      0.00      0.00         2
       Water       0.22      0.36      0.28        11

    accuracy              

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
